In [3]:
import config
import requests
import datetime as dt
import json
from datetime import datetime as dt, date
from dateutil.relativedelta import relativedelta
import pandas as pd
import requests
import os
import sys

API_KEY = config.API_KEY

# Symbol lists contain all the stocks that we are interested in. 
symbol_list = ['AAPL','MSFT','GOOG','AMZN','NVDA','BRK-B','TSLA','META','JNJ','V']

In [4]:

def get_daily_prices(symbol):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&outputsize=full&apikey={API_KEY}'
    r = requests.get(url)
    
    data = r.json()

    if r.status_code == 200:
       
        price_data = data["Time Series (Daily)"]

        dates = []
        full_dates = list(price_data.keys())
    
        today = date(2020,2,1)
   
    
        prices = []
        for i in range(0, len(full_dates)):
            day = dt.strptime(full_dates[i], "%Y-%m-%d").date()
            if day < today - relativedelta(years=10):
                break
            else:
                dates.append(full_dates[i])
                prices.append(float(price_data[full_dates[i]]["4. close"]))

        his_prices = {"dates": dates, "prices": prices}
    
        return his_prices
    else: return None

get_daily_prices('IBM')

{'dates': ['2023-04-21',
  '2023-04-20',
  '2023-04-19',
  '2023-04-18',
  '2023-04-17',
  '2023-04-14',
  '2023-04-13',
  '2023-04-12',
  '2023-04-11',
  '2023-04-10',
  '2023-04-06',
  '2023-04-05',
  '2023-04-04',
  '2023-04-03',
  '2023-03-31',
  '2023-03-30',
  '2023-03-29',
  '2023-03-28',
  '2023-03-27',
  '2023-03-24',
  '2023-03-23',
  '2023-03-22',
  '2023-03-21',
  '2023-03-20',
  '2023-03-17',
  '2023-03-16',
  '2023-03-15',
  '2023-03-14',
  '2023-03-13',
  '2023-03-10',
  '2023-03-09',
  '2023-03-08',
  '2023-03-07',
  '2023-03-06',
  '2023-03-03',
  '2023-03-02',
  '2023-03-01',
  '2023-02-28',
  '2023-02-27',
  '2023-02-24',
  '2023-02-23',
  '2023-02-22',
  '2023-02-21',
  '2023-02-17',
  '2023-02-16',
  '2023-02-15',
  '2023-02-14',
  '2023-02-13',
  '2023-02-10',
  '2023-02-09',
  '2023-02-08',
  '2023-02-07',
  '2023-02-06',
  '2023-02-03',
  '2023-02-02',
  '2023-02-01',
  '2023-01-31',
  '2023-01-30',
  '2023-01-27',
  '2023-01-26',
  '2023-01-25',
  '2023-01-24',

In [108]:
# A template method for showing one data for all stocks

df_incremental = []
for symbol in symbol_list:
    temp = get_daily_prices(symbol)
    temp2 = dict()
    temp2["date"] = temp["dates"]
    temp2[symbol] = temp["prices"]
    temp_df = pd.DataFrame.from_dict(temp2)
    df_incremental.append(temp_df)

list_of_dates = [df_incremental[i]["date"].tolist() for i in range(len(df_incremental))]
unique_dates = sorted(list(set([item for sublist in list_of_dates for item in sublist])))
df= pd.DataFrame(unique_dates, columns=["date"])

for i in range(0, len(df_incremental)):
    df = pd.merge(df, df_incremental[i], on="date")

#df # also works if there are missing dates for some stocks even first one
df['date'] = pd.to_datetime(df['date'])

# Set the date column as the index
df.set_index('date', inplace=True)

# Resample the data to weekly frequency and calculate the mean price
df_weekly = df.resample('W').mean()  # W for weekly, M for monthly, A for annual

# Print the weekly data
print(df_weekly)


                  AAPL      MSFT      GOOG      AMZN      NVDA     BRK-B  \
date                                                                       
2014-03-30  537.160000   39.8300  559.2250  338.3800   17.8450  123.5000   
2014-04-06  538.310000   40.9280  560.8020  335.5870   18.3940  124.5060   
2014-04-13  524.064000   39.7318  545.7480  321.0950   18.4820  122.8460   
2014-04-20  520.897475   39.8350  540.4000  320.1450   18.4550  124.8100   
2014-04-27  545.465800   39.8780  526.3420  325.1500   18.9320  127.0120   
...                ...       ...       ...       ...       ...       ...   
2023-02-26  148.375000  252.0425   91.0675   94.9225  220.8975  303.2425   
2023-03-05  147.516000  250.4500   91.4480   93.4380  233.2380  306.9320   
2023-03-12  151.478000  253.1260   93.6140   92.8400  234.8480  309.5820   
2023-03-19  153.380000  267.1560   97.1980   96.5000  245.0460  300.3220   
2023-03-26  158.738000  275.3060  104.8620   98.7720  265.0740  300.3860   

           

In [7]:
def get_daily_adjusted_prices(symbol):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&outputsize=full&apikey={API_KEY}'
    
    r = requests.get(url)
    #data_adjusted = r.json()
    data = r.json()
  
    if   r.status_code==200:
       
        #price_data_adj = data_adjusted["Time Series (Daily)"]
        price_data = data["Time Series (Daily)"]
        price_df = pd.DataFrame(price_data).transpose().reset_index()
        price_df.rename(columns={'index':'Date'},inplace=True)
        #price_df_adj = pd.DataFrame(price_data_adj).reset_index()
        #price_df_adj.rename(columns={'index':'Date'},inplace=True)
        #price_df = price_df.merge(price_df_adj.loc[:,["Date","5. adjusted close"]], on='Date',how='left')
        price_df.loc[:,'Date'] = [dt.strptime(d,'%Y-%m-%d') for d in price_df.Date]
        end_date = dt(2020,2,1)
        start_date = dt(2010,1,1)
        price_df = price_df.loc[(price_df.Date>=start_date)&(price_df.Date<=end_date),:].reset_index(drop=True)
        return price_df
    else: return None

def get_monthly_adjusted_prices(symbol):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY_ADJUSTED&symbol={symbol}&outputsize=full&apikey={API_KEY}'
    
    r = requests.get(url)
    data = r.json()
  
    if   r.status_code==200:
        price_data = data["Monthly Adjusted Time Series"]
        price_df = pd.DataFrame(price_data).transpose().reset_index()
        price_df.rename(columns={'index':'Date'},inplace=True)

        price_df.loc[:,'Date'] = [dt.strptime(d,'%Y-%m-%d') for d in price_df.Date]
        end_date = dt(2020,2,1)
        start_date = dt(2010,1,1)
        price_df = price_df.loc[(price_df.Date>=start_date)&(price_df.Date<=end_date),:].reset_index(drop=True)
        return price_df
    else: return None

def get_weekly_adjusted_prices(symbol):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY_ADJUSTED&symbol={symbol}&outputsize=full&apikey={API_KEY}'
    
    r = requests.get(url)
    data = r.json()
  
    if   r.status_code==200:
        price_data = data["Weekly Adjusted Time Series"]
        price_df = pd.DataFrame(price_data).transpose().reset_index()
        price_df.rename(columns={'index':'Date'},inplace=True)

        price_df.loc[:,'Date'] = [dt.strptime(d,'%Y-%m-%d') for d in price_df.Date]
        end_date = dt(2020,2,1)
        start_date = dt(2010,1,1)
        price_df = price_df.loc[(price_df.Date>=start_date)&(price_df.Date<=end_date),:].reset_index(drop=True)
        return price_df
    else: return None

In [6]:
get_monthly_adjusted_prices("IBM")

,Date,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount
0,2020-01-31 00:00:00,135.0000,145.7900,133.2000,143.7300,116.9057,112864967,0.0000
1,2019-12-31 00:00:00,134.4500,136.4200,130.6900,134.0400,109.0241,71661951,0.0000
2,2019-11-29 00:00:00,134.5000,139.1400,132.7500,134.4500,109.3576,64898948,1.6200
3,2019-10-31 00:00:00,145.5900,147.3500,130.9000,133.7300,107.5071,92647927,0.0000
4,2019-09-30 00:00:00,134.8500,146.5700,133.3300,145.4200,116.9048,62083780,0.0000
...,...,...,...,...,...,...,...,...
116,2010-05-28 00:00:00,129.3900,133.1000,116.0000,125.2600,76.6183,188444200,0.6500
117,2010-04-30 00:00:00,128.9500,132.2800,127.1200,129.0000,78.4943,146426400,0.0000
118,2010-03-31 00:00:00,127.5000,130.7300,125.2000,128.2500,78.0379,139997300,0.0000
119,2010-02-26 00:00:00,123.2300,128.2700,121.6100,127.1600,77.3747,113211200,0.5500


In [8]:
for stock in symbol_list:
    stock_daily = get_daily_adjusted_prices(symbol)
    stock_daily.to_csv("Daily\\"+stock+'.csv',index=0)
    stock_weekly = get_weekly_adjusted_prices("META")
    stock_weekly.to_csv("Weekly\\"+stock+'.csv',index=0)
    stock_monthly = get_monthly_adjusted_prices("META")
    stock_monthly.to_csv("Monthly\\"+stock+'.csv',index=0)
    

NameError: name 'stock_df' is not defined